In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from itertools import combinations

En este notebook voy a explorar diferentes formas de implementar el problema antes de diseñar el archivo final.

# Formalización del problema

Antes de empezar a programar voy a intentar establecer un modelo el cual voy a intentar implementar.

Básicamente tenemos una load, $L$, y una serie de centrales de energía. Denotemos por $P_i$ la energía que genera la central $i$ donde $i = 1,...,n$ con $n$ el número total de centrales de energía que tenemos disponibles. En todo momento la energía generada por todas las plantas debe igualar la load:
$$
L = e_1 P_1 + ... + e_n P_n
$$
donde $e_i$ es la correspondiente eficiencia de la fuente de energía.
El objetivo es minimizar el coste de la energía, con lo cual denotemos por $c_i$ el coste de generar energía en la planta $i$. Por tanto, nuestra función objetivo es:
$$
c_1 P_1 + ... + c_n P_n
$$
Hay tres tipos de centrales de energía según cómo generen la energía: 

1. Usando un turbojet (eficiencia del $30\%$)
2. Central de gas (eficiencia del $50\%$)
3. Usando molinos de viento (eficiencia depende del viento que haga, en un principio)

Cada una de estas centrales de energía genera energía con un cierto coste, lo que nos permitirá calcular los $c_i$. Además, hay unos límites a la energía que podemos generar dados por los valores $Pmax_i$ y $Pmin_i$:

$$
Pmin_i \leq P_i \leq Pmax_i
$$

En general, solo las centrales de gas tienen un $Pmin$ por lo que dice el enunciado.

Con lo cual podemos plantear el problema al que nos vamos a enfrentar como un problema de programación lineal:

$$
\min \quad c_1 P_1 + ... + c_n P_n
$$
$$
s.t. \quad L = e_1 P_1 + ... + e_n P_n
$$
$$
\quad \, \, Pmin_i \leq P_i \leq Pmax_i \quad \forall i
$$

Así pues, una solución sistemática del problema requiere:

1. Implementar un algoritmo que resuelva problemas de programación lineal.
2. Calcular los costes de la energía para un problema concreto.
3. Plantear el correspondiente problema de programación lineal y resolverlo.

# Lectura de datos

Para empezar voy a comprobar cómo abrir y guardar archivos .json.

In [2]:
file = open('example_payloads\\payload1.json')
data = json.load(file)
file.close()

In [3]:
data

{'load': 480,
 'fuels': {'gas(euro/MWh)': 13.4,
  'kerosine(euro/MWh)': 50.8,
  'co2(euro/ton)': 20,
  'wind(%)': 60},
 'powerplants': [{'name': 'gasfiredbig1',
   'type': 'gasfired',
   'efficiency': 0.53,
   'pmin': 100,
   'pmax': 460},
  {'name': 'gasfiredbig2',
   'type': 'gasfired',
   'efficiency': 0.53,
   'pmin': 100,
   'pmax': 460},
  {'name': 'gasfiredsomewhatsmaller',
   'type': 'gasfired',
   'efficiency': 0.37,
   'pmin': 40,
   'pmax': 210},
  {'name': 'tj1',
   'type': 'turbojet',
   'efficiency': 0.3,
   'pmin': 0,
   'pmax': 16},
  {'name': 'windpark1',
   'type': 'windturbine',
   'efficiency': 1,
   'pmin': 0,
   'pmax': 150},
  {'name': 'windpark2',
   'type': 'windturbine',
   'efficiency': 1,
   'pmin': 0,
   'pmax': 36}]}

El archivo resultante es un diccionario de python. Vamos a comprobar cómo hay que almacenar la respuesta:

In [4]:
file = open('example_payloads\\response3.json')
data = json.load(file)
file.close()

In [5]:
data

[{'name': 'windpark1', 'p': 90.0},
 {'name': 'windpark2', 'p': 21.6},
 {'name': 'gasfiredbig1', 'p': 460.0},
 {'name': 'gasfiredbig2', 'p': 338.4},
 {'name': 'gasfiredsomewhatsmaller', 'p': 0.0},
 {'name': 'tj1', 'p': 0.0}]

El formato es simplemente un array que almacena diccionarios, cada uno con el nombre de la central y la cantidad de energía que genera. Vamos a reproducir tal array:

In [6]:
data = [{'name': 'windpark1', 'p': 90.0},
 {'name': 'windpark2', 'p': 21.6},
 {'name': 'gasfiredbig1', 'p': 460.0},
 {'name': 'gasfiredbig2', 'p': 338.4},
 {'name': 'gasfiredsomewhatsmaller', 'p': 0.0},
 {'name': 'tj1', 'p': 0.0}]

data

[{'name': 'windpark1', 'p': 90.0},
 {'name': 'windpark2', 'p': 21.6},
 {'name': 'gasfiredbig1', 'p': 460.0},
 {'name': 'gasfiredbig2', 'p': 338.4},
 {'name': 'gasfiredsomewhatsmaller', 'p': 0.0},
 {'name': 'tj1', 'p': 0.0}]

Y lo guardamos:

In [7]:
save_file = open("respuestas\\ejemplo1.json", "w")
json.dump(data, save_file)  
save_file.close()  

Podemos comprobar si tal archivo está guardado correctamente o no:

In [8]:
file = open('respuestas\\ejemplo1.json')
data = json.load(file)
file.close()

In [9]:
data

[{'name': 'windpark1', 'p': 90.0},
 {'name': 'windpark2', 'p': 21.6},
 {'name': 'gasfiredbig1', 'p': 460.0},
 {'name': 'gasfiredbig2', 'p': 338.4},
 {'name': 'gasfiredsomewhatsmaller', 'p': 0.0},
 {'name': 'tj1', 'p': 0.0}]

Con esto hemos comprobado el correcto manejo de archivos .json con lo cual ya podemos proceder a la siguiente sección.

# Algoritmo de programación lineal

Vamos a implementar un algorithmo que permita resolver problemas de programación lineal como los que nos vamos a enfrentar. Con tiempo infinito sería posible implementar el algoritmo simplex de modo que la resolución sea exacta, precisa y eficiente. Sin embargo, debido a las restricciones temporales vamos a tener que ingeniar un método alternativo. Para cada planta de energía vamos a introducir dos variables: $s_i$ y $f_i$, que representan la cantidad de energía que sobra con respecto a $Pmin_i$ y la cantidad de energía que falta para llegar a $Pmax_i$, de modo que las desigualdades se pueden expresar como:

$$
Pmin_i = P_i - s_i \quad \quad \quad Pmax_i = P_i + f_i
$$

Así todas las restricciones son de igualdad. Si tenemos seis centrales de energía, el número total de restricciones sería $13$. Podemos calcular el rango de la matriz de restricciones, que es $n \leq 13$, de modo que la solución del problema la encontraríamos tomando $13 - n$ variables iguales a cero y el resto de las variables se podrían calcular por medio de invertir una matriz de rango $n$. Debido al tamaño de los ejemplos podemos calcular todas las posibles matrices de estas características y mediante búsqueda finita podemos calcular todas las soluciones posibles y extraer la óptima. Es una solución tosca y no se puede escalar para el caso de que haya muchas centrales debido a que tendríamos que invertir $C_{m, n}$ matrices donde $m$ es el número total de centrales y $n$ es el rango de la matriz, pero dadas las restricciones resulta más fácil de implementar que la alternativa y vale para el contexto dado. Así pues, vamos a implementar esta solución:

In [10]:
def algoritmo(A, c, b, centrales):
    total = 3*centrales
    
    #Almacenamos el rango de la matriz
    rango = 2*centrales + 1
    #Calculamos los índices de las columnas de todas las posibles submatrices
    indices = np.array([[True if i in comb else False for i in range(total)] for comb in combinations(np.arange(total), rango)])
    nrows = np.shape(indices)[0]
        
    solucion = {}
    
    for i in range(nrows):
        #Escogemos la submatriz correspondiente
        submatriz = A[:,indices[i,:]]
            
        try:
            #Tratamos de resolver el sistema correspondiente
            z = np.linalg.solve(submatriz, b)
    
            #La solución la insertamos en un vector lleno de ceros
            sol_real = np.zeros(3*n)
            sol_real[indices[i,:]] = z
            value = c.dot(sol_real)
    
            #Descartamos soluciones con componentes negativas
            if np.sum(sol_real[sol_real != 0] < 0) == 0:
                solucion[i] = {'x': sol_real, 'obj': c.dot(sol_real)}
            else:
                solucion[i] = {'x': None, 'obj': 1e99}
        except:
            #Si surge algún problema devolvemos una solución nula
            solucion[i] = {'x': None, 'obj': 1e99}
    
    #Guardamos los valores obtenidos y buscamos el mínimo
    obj = np.array([solucion[i]['obj'] for i in range(nrows)])
    opt_index = np.argmin(obj)
    #Devolvemos la solución
    return {'x': solucion[opt_index]['x'], 'obj': solucion[opt_index]['obj']}

# Ejemplo de resolución

Estamos preparados para dar un ejemplo de cómo resolver el problema antes de hacer el archivo final.

In [11]:
#Cargamos un ejemplo
file = open('example_payloads\\payload1.json')
data = json.load(file)
file.close()

In [12]:
#Guardamos los datos

L = data['load']
fuels = data['fuels']
powerplants = data['powerplants']

n = len(powerplants)

names = [powerplant['name'] for powerplant in powerplants]
types = [powerplant['type'] for powerplant in powerplants]
efficiency = [powerplant['efficiency'] for powerplant in powerplants]
pmin = [powerplant['pmin'] for powerplant in powerplants]
pmax = [powerplant['pmax'] for powerplant in powerplants]

In [13]:
#Calculamos el vector de costes
costs = np.array([0 for i in range(n)])
types = np.array(types)
costs[types == 'gasfired'] = fuels['gas(euro/MWh)']
costs[types == 'turbojet'] = fuels['kerosine(euro/MWh)']

#Vamos a considerar que la eficiencia de una planta eólica es el viento que hace
efficiency = np.array(efficiency)
efficiency[types == 'windturbine'] = fuels['wind(%)']/100 

In [14]:
#Añadimos las variables de relajación en el vector de costes
c = np.concatenate((costs, np.zeros(2*n)))

#Calculamos el término independiente de las restricciones
b = [L] + pmin + pmax
b = np.array(b)

#Calculamos la matriz de restricciones

#Primera fila: vector de eficiencias
first_row = np.concatenate((efficiency, np.zeros(2*n)))
first_row = np.reshape(first_row, (1, 3*n))

#El resto de filas es fácil definirlas por bloques
#primeras n columnas: variables "reales", P_i
#siguientes n columnas: variables "pmin", s_i
#últimas n columnas: variables "pmax", f_i
identity = np.eye(n)
zeros = np.zeros((n,n))

pmin_block = np.concatenate((identity, -identity, zeros), axis = 1)
pmax_block = np.concatenate((identity, zeros, identity), axis = 1)

A = np.concatenate((first_row, pmin_block, pmax_block), axis = 0)

In [15]:
#Resolvemos
sol = algoritmo(A, c, b, n)

In [16]:
#Expresamos la solución adecuadamente
respuesta = [{'name': names[i], 'p': sol['x'][i]} for i in range(n)]

In [17]:
#Guardamos la solución en el formato correcto
save_file = open("respuestas\\solucion_payload1.json", "w")
json.dump(data, save_file)  
save_file.close()  

# Archivo definitivo

Ahora estamos preparados para desarrollar un archivo definitivo tal y como se pide en el enunciado a partir del código desarrollado. Para empezar vamos a crear un archivo requirements.txt

In [18]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


Ahora podemos construir el REST API requerido. No estoy familiarizado con estas herramientas por lo cual voy a experimentar un poco antes de crear el archivo final.


# Correcciones

Esta primera versión es primitiva y ya permite obtener soluciones tangibles. No obstante, hay un problema fundamental: las centrales para las que pmin es mayor que cero no funcionan como deberían. La cuestión es que la energía mínima que se genera en dichas centrales no es pmin: es $0$. SI se enciende la central, entonces tenemos que generar como mínimo pmin. Pero si no fuera rentable encenderla podemos simplemente no generar energía, con lo que generaríamos $0$ en lugar de pmin. Tal como está planteado nuestro problema, como mínimo siempre vamos a generar pmin, lo cual es un error. Una forma de proceder es la siguiente:

1. Calcular la solución óptima con el algoritmo previo
2. Probar todas las combinaciones de estaciones de gas que se puedan cerrar a la vez
3. Para cada combinación, calcular la solución óptima del correspondiente problema de optimización
4. Usar como solución aquella que, entre todas las soluciones obtenidas, resulte en el mínimo para la función objetivo

Voy a intentar implementar este procedimiento en el tiempo que me queda, pero en caso de no poder por lo menos queda expuesta una idea sobre cómo mejorar mi solución.

In [55]:
def run(case):
    #Guardamos los datos
    L = data['load']
    fuels = data['fuels']
    powerplants = data['powerplants']
        
    n = len(powerplants)
        
    names = [powerplant['name'] for powerplant in powerplants]
    types = [powerplant['type'] for powerplant in powerplants]
    efficiency = [powerplant['efficiency'] for powerplant in powerplants]
    pmin = [powerplant['pmin'] for powerplant in powerplants]
    pmax = [powerplant['pmax'] for powerplant in powerplants]
    
    #Calculamos el vector de costes
    costs = np.array([0 for i in range(n)])
    types = np.array(types)
    costs[types == 'gasfired'] = fuels['gas(euro/MWh)']
    costs[types == 'turbojet'] = fuels['kerosine(euro/MWh)']
        
    #Vamos a considerar que la eficiencia de una planta eólica es el viento que hace
    efficiency = np.array(efficiency)
    efficiency[types == 'windturbine'] = fuels['wind(%)']/100 


    pmin = np.array(pmin)
    pmax = np.array(pmax)
    pmin[types == 'gasfired'] = case*pmin[types == 'gasfired']
    pmax[types == 'gasfired'] = case*pmax[types == 'gasfired']
    
    #Añadimos las variables de relajación en el vector de costes
    c = np.concatenate((costs, np.zeros(2*n)))
        
    #Calculamos el término independiente de las restricciones
    b = [L] + pmin + pmax
    b = np.array(b)
        
    #Calculamos la matriz de restricciones
        
    #Primera fila: vector de eficiencias
    first_row = np.concatenate((efficiency, np.zeros(2*n)))
    first_row = np.reshape(first_row, (1, 3*n))
        
    #El resto de filas es fácil definirlas por bloques
    #primeras n columnas: variables "reales", P_i
    #siguientes n columnas: variables "pmin", s_i
    #últimas n columnas: variables "pmax", f_i
    identity = np.eye(n)
    zeros = np.zeros((n,n))
        
    pmin_block = np.concatenate((identity, -identity, zeros), axis = 1)
    pmax_block = np.concatenate((identity, zeros, identity), axis = 1)
        
    A = np.concatenate((first_row, pmin_block, pmax_block), axis = 0)
    
    def algoritmo(A, b, c, n):
        #Resolvemos
        total = 3*n
            
        #Almacenamos el rango de la matriz
        rango = 2*n + 1
        #Calculamos los índices de las columnas de todas las posibles submatrices
        indices = np.array([[True if i in comb else False for i in range(total)] for comb in combinations(np.arange(total), rango)])
        nrows = np.shape(indices)[0]
                
        solucion = {}
            
        for i in range(nrows):
            #Escogemos la submatriz correspondiente
            submatriz = A[:,indices[i,:]]
                    
            try:
                #Tratamos de resolver el sistema correspondiente
                z = np.linalg.solve(submatriz, b)
                
                #La solución la insertamos en un vector lleno de ceros
                sol_real = np.zeros(3*n)
                sol_real[indices[i,:]] = z
                value = c.dot(sol_real)
                
                #Descartamos soluciones con componentes negativas
                if np.sum(sol_real[sol_real != 0] < 0) == 0:
                    solucion[i] = {'x': sol_real, 'obj': c.dot(sol_real)}
                else:
                    solucion[i] = {'x': None, 'obj': 1e99}
            except:
                #Si surge algún problema devolvemos una solución nula
                solucion[i] = {'x': None, 'obj': 1e99}
            
        #Guardamos los valores obtenidos y buscamos el mínimo
        obj = np.array([solucion[i]['obj'] for i in range(nrows)])
        opt_index = np.argmin(obj)
        #Devolvemos la solución
        sol = {'x': solucion[opt_index]['x'], 'obj': solucion[opt_index]['obj']}
        return sol
        
    sol = algoritmo(A, b, c, n)
    #Expresamos la solución adecuadamente
    try:
        respuesta = [{'name': names[i], 'p': sol['x'][i]} for i in range(n)] +  [{'obj': sol['obj']}]
    except:
        respuesta = [{'name': names[i], 'p': 1e99} for i in range(n)] + [{'obj': sol['obj']}]
    return respuesta


In [85]:
import itertools
repeat = np.sum(types == 'gasfired')
lst = [list(i) for i in itertools.product([0, 1], repeat=repeat)]

sol = {}
i = 0

for case in lst:
    sol[i] = run(np.array(case))
    i = i + 1 

In [86]:
obj = [sol[i][-1]['obj'] for i in range(8)]
opt_index = np.argmin(obj)
solution = sol[opt_index][1:n]

In [88]:
sol

{0: [{'name': 'gasfiredbig1', 'p': 1e+99},
  {'name': 'gasfiredbig2', 'p': 1e+99},
  {'name': 'gasfiredsomewhatsmaller', 'p': 1e+99},
  {'name': 'tj1', 'p': 1e+99},
  {'name': 'windpark1', 'p': 1e+99},
  {'name': 'windpark2', 'p': 1e+99},
  {'obj': 1e+99}],
 1: [{'name': 'gasfiredbig1', 'p': 1e+99},
  {'name': 'gasfiredbig2', 'p': 1e+99},
  {'name': 'gasfiredsomewhatsmaller', 'p': 1e+99},
  {'name': 'tj1', 'p': 1e+99},
  {'name': 'windpark1', 'p': 1e+99},
  {'name': 'windpark2', 'p': 1e+99},
  {'obj': 1e+99}],
 2: [{'name': 'gasfiredbig1', 'p': 1e+99},
  {'name': 'gasfiredbig2', 'p': 1e+99},
  {'name': 'gasfiredsomewhatsmaller', 'p': 1e+99},
  {'name': 'tj1', 'p': 1e+99},
  {'name': 'windpark1', 'p': 1e+99},
  {'name': 'windpark2', 'p': 1e+99},
  {'obj': 1e+99}],
 3: [{'name': 'gasfiredbig1', 'p': 1e+99},
  {'name': 'gasfiredbig2', 'p': 1e+99},
  {'name': 'gasfiredsomewhatsmaller', 'p': 1e+99},
  {'name': 'tj1', 'p': 1e+99},
  {'name': 'windpark1', 'p': 1e+99},
  {'name': 'windpark2', 

Ya he excedido las cuatro horas así que voy a terminar aquí. El código de arriba en un principio debería poder forzar a las variables del gas que sean cero cada vez. Esto permitiría simular que la fábrica no se abre. Cada caso debería corresponderse con considerar si se abre o no cada fábrica de gas. No obstante, hay algún fallo que no he podido encontrar, de modo que la resolución queda a medias pero es fácilmente solventable. La implementación de un archivo Docker se escapa de mis conocimientos por lo cual necesitaría más tiempo para implementarla (como me ocurrió con el caso de la REST API: si ya hubiera sabido manejarla podría haber sacado más tiempo para completar satisfactoriamente la implementación). No obstante, el segundo objetivo secundario es fácilmente implementable: todo lo que hay que hacer es incluir una serie de constraints para las gas-fired powerplants. Estimo que como mucho en media hora podría incluirlo, pero no lo haré para no seguir sobrepasando el límite de tiempo.